In [3]:
!pip install fastapi

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.2 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)


In [5]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pymysql


  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from fastapi import APIRouter, HTTPException, Query
from pydantic import BaseModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from typing import List, Optional, Dict, Any
from datetime import datetime
import traceback
from sqlalchemy import text

# 既存のモジュールをインポート
from db_control import crud
from db_control.connect_MySQL import engine
from db_control.mymodels_MySQL import User, UserTag, Tag, PointTransaction, Event, EventTag, Store
from db_control.crud import session_scope

platform uname_result(system='Darwin', node='iwamurahayatonoMacBook-Air.local', release='22.1.0', version='Darwin Kernel Version 22.1.0: Sun Oct  9 20:14:30 PDT 2022; root:xnu-8792.41.9~2/RELEASE_ARM64_T8103', machine='arm64')
DB_PASSWORD before encoding: tech0-himdb
Encoded DB_PASSWORD: tech0-himdb
2025-04-16 05:31:12,046 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-04-16 05:31:12,047 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 05:31:12,230 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-04-16 05:31:12,230 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-16 05:31:12,318 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-04-16 05:31:12,318 INFO sqlalchemy.engine.Engine [raw sql] {}
Secure connection successful with SQLAlchemy!


In [9]:
# レスポンスモデルの定義
class EventRecommendation(BaseModel):
    id: str
    imageUrl: Optional[str] = None
    area: Optional[str] = None
    title: str
    date: Optional[str] = None
    tags: List[str] = []
    description: Optional[str] = None
    points: Optional[int] = None

In [10]:
class RecommendationResponse(BaseModel):
    events: List[EventRecommendation]
    similarUsers: List[int] = []

In [11]:
# データ取得関数
def get_user_data(session) -> pd.DataFrame:
    """ユーザーデータを取得し前処理する"""
    query_users = text("""
    SELECT user_id, gender, relationship_id, postal_code, birth_date 
    FROM Users
    """)
    result_users = session.execute(query_users).fetchall()
    if not result_users:
        raise HTTPException(status_code=404, detail="ユーザーデータが見つかりません")
    
    # 結果をDataFrameに変換
    df_users = pd.DataFrame(result_users)
    
    # 性別を数値に変換
    df_users["gender"] = df_users["gender"].map({"M": 0, "F": 1, "U": 2})
    
    # 年齢の計算
    df_users["age"] = pd.to_datetime("today").year - pd.to_datetime(df_users["birth_date"]).dt.year
    df_users.drop(columns=["birth_date"], inplace=True)
    
    # 年齢のMin-Maxスケーリング
    scaler = MinMaxScaler()
    df_users["age"] = scaler.fit_transform(df_users[["age"]])
    
    # 郵便番号をワンホットエンコーディング
    df_users = pd.get_dummies(df_users, columns=["postal_code"])
    return df_users

実行テスト

In [19]:
with session_scope() as session:
    df_users = get_user_data(session)

# データ確認
print(df_users.shape)  # 行数と列数
print(df_users.columns)  # カラム名一覧
df_users.head()  # 先頭5件

2025-04-16 05:41:43,689 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 05:41:43,702 INFO sqlalchemy.engine.Engine 
    SELECT user_id, gender, relationship_id, postal_code, birth_date 
    FROM Users
    
2025-04-16 05:41:43,703 INFO sqlalchemy.engine.Engine [generated in 0.00185s] {}
2025-04-16 05:41:44,039 INFO sqlalchemy.engine.Engine COMMIT
(30, 4)
Index(['user_id', 'gender', 'relationship_id', 'age'], dtype='object')


,user_id,gender,relationship_id,age
0,1,0,1,0.890625
1,2,0,1,0.953125
2,3,0,1,1.000000
3,4,1,2,0.859375
4,5,1,2,0.984375


In [20]:
def get_user_tags(session) -> pd.DataFrame:
    """ユーザータグデータを取得しワンホットエンコーディングする"""
    query_tags = text("""
    SELECT u.user_id, t.tag_name  
    FROM UserTags u 
    JOIN Tags t ON u.tag_id = t.tag_id
    """)
    result_tags = session.execute(query_tags).fetchall()
    
    # タグのワンホットエンコーディング
    if result_tags:
        df_tags = pd.DataFrame(result_tags)
        df_tags_onehot = df_tags.pivot_table(
            index="user_id", columns="tag_name", aggfunc="size", fill_value=0
        )
        return df_tags_onehot.add_prefix("tag_")
    else:
        # タグがない場合は空のDataFrameを作成
        return pd.DataFrame()

In [21]:
with session_scope() as session:
    df_tags_onehot = get_user_tags(session)

# データ確認
print(df_tags_onehot.shape)  # 行数と列数
print(df_tags_onehot.columns)  # カラム名一覧
df_tags_onehot.head()  # 先頭5件

2025-04-16 05:54:00,665 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 05:54:00,676 INFO sqlalchemy.engine.Engine 
    SELECT u.user_id, t.tag_name  
    FROM UserTags u 
    JOIN Tags t ON u.tag_id = t.tag_id
    
2025-04-16 05:54:00,676 INFO sqlalchemy.engine.Engine [generated in 0.00159s] {}
2025-04-16 05:54:01,093 INFO sqlalchemy.engine.Engine COMMIT
(30, 26)
Index(['tag_お祭り', 'tag_のんびり派', 'tag_アウトドア', 'tag_アクティブ', 'tag_アート',
       'tag_ウォーキング', 'tag_エコ志向', 'tag_エンタメ', 'tag_カフェ巡り', 'tag_グルメ',
       'tag_スイーツ', 'tag_スポーツ', 'tag_デジタル', 'tag_ボランティア', 'tag_ライフスタイル',
       'tag_ライブイベント', 'tag_伝統工芸', 'tag_健康志向', 'tag_地元グルメ', 'tag_地域活性化',
       'tag_子育て支援', 'tag_学び・体験', 'tag_文化・歴史', 'tag_社会貢献', 'tag_神社仏閣',
       'tag_食べ歩き'],
      dtype='object', name='tag_name')


tag_name,tag_お祭り,tag_のんびり派,tag_アウトドア,tag_アクティブ,tag_アート,tag_ウォーキング,tag_エコ志向,tag_エンタメ,tag_カフェ巡り,tag_グルメ,...,tag_伝統工芸,tag_健康志向,tag_地元グルメ,tag_地域活性化,tag_子育て支援,tag_学び・体験,tag_文化・歴史,tag_社会貢献,tag_神社仏閣,tag_食べ歩き
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
def get_transaction_data(session) -> pd.DataFrame:
    """ポイント取引データを取得しワンホットエンコーディングする"""
    query_transactions = text("""
    SELECT user_id, store_id 
    FROM PointTransaction 
    WHERE user_id IS NOT NULL
    """)
    result_transactions = session.execute(query_transactions).fetchall()
    
    # 取引のワンホットエンコーディング
    if result_transactions:
        df_transactions = pd.DataFrame(result_transactions)
        df_transactions_onehot = df_transactions.pivot_table(
            index="user_id", columns="store_id", aggfunc="size", fill_value=0
        )
        return df_transactions_onehot.add_prefix("store_")
    else:
        # 取引がない場合は空のDataFrameを作成
        return pd.DataFrame()


In [23]:
with session_scope() as session:
    df_transactions_onehot = get_transaction_data(session)

# データ確認
print(df_transactions_onehot.shape)  # 行数と列数
print(df_transactions_onehot.columns)  # カラム名一覧
df_transactions_onehot.head()  # 先頭5件

2025-04-16 05:56:12,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 05:56:12,023 INFO sqlalchemy.engine.Engine 
    SELECT user_id, store_id 
    FROM PointTransaction 
    WHERE user_id IS NOT NULL
    
2025-04-16 05:56:12,023 INFO sqlalchemy.engine.Engine [generated in 0.00040s] {}
2025-04-16 05:56:12,141 INFO sqlalchemy.engine.Engine COMMIT
(22, 6)
Index(['store_1', 'store_2', 'store_3', 'store_4', 'store_5', 'store_6'], dtype='object', name='store_id')


store_id,store_1,store_2,store_3,store_4,store_5,store_6
user_id,,,,,,
1,5,0,0,0,0,0
2,0,3,0,0,0,0
3,3,0,3,0,1,0
4,1,0,0,3,1,0
5,0,0,0,1,3,1


In [26]:
def get_favorite_events_onehot(session) -> pd.DataFrame:
    """お気に入りイベントをワンホットエンコーディングする"""
    query_favorites = text("""
    SELECT user_id, event_id
    FROM FavoriteEvents
    """)
    result_fav = session.execute(query_favorites).fetchall()
    
    if result_fav:
        df_fav = pd.DataFrame(result_fav)
        df_fav_onehot = df_fav.pivot_table(
            index="user_id", columns="event_id", aggfunc="size", fill_value=0
        )
        return df_fav_onehot.add_prefix("fav_event_")
    else:
        return pd.DataFrame()


In [28]:
with session_scope() as session:
    df_fav_onehot = get_favorite_events_onehot(session)

# データ確認
print(df_fav_onehot.shape)  # 行数と列数
print(df_fav_onehot.columns)  # カラム名一覧
df_fav_onehot.head()  # 先頭5件

2025-04-16 06:03:18,892 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 06:03:18,898 INFO sqlalchemy.engine.Engine 
    SELECT user_id, event_id
    FROM FavoriteEvents
    
2025-04-16 06:03:18,899 INFO sqlalchemy.engine.Engine [generated in 0.00147s] {}
2025-04-16 06:03:19,006 INFO sqlalchemy.engine.Engine COMMIT
(1, 3)
Index(['fav_event_75', 'fav_event_96', 'fav_event_98'], dtype='object', name='event_id')


event_id,fav_event_75,fav_event_96,fav_event_98
user_id,,,
3,1,1,1
